In [ ]:
import numpy as np
import pandas as pd
import json
import requests
import urllib.parse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

### Test dengan data API

In [ ]:
API_URL = "https://api.goapi.io/stock/idx/prices"
API_KEY = '833b2587-34b1-5172-801a-6be0cf21'
SYMBOL_URL = "https://api.goapi.io/stock/idx/trending"

In [ ]:
def generate_saham_tren(api_url, api_key):
  API_URL_KODE = api_url
  headers = {"X-API-KEY": api_key}

  response = requests.get(API_URL_KODE, headers=headers).json()
  response_str = json.dumps(response)
  SYMBOL = pd.json_normalize(json.loads(response_str), record_path=['data', 'results'])
  SYMBOL = SYMBOL['symbol']

  SYMBOL_string = ','.join(SYMBOL)

  return SYMBOL_string

In [ ]:
SYMBOL_string = generate_saham_tren(SYMBOL_URL, API_KEY)

In [ ]:
SYMBOL_string

'GOTO,GZCO,BRPT,BIPI,PGEO,NATO,ARCI,APIC,MEDC,TPIA,BBRI,BMRI,TLKM,AMMN,ADMR,BREN,BBNI,AKRA,BBCA,PANI,ADRO,ANTM,ARTO,MDKA,AMRT,PTRO,ASII,MIKA,INCO,FILM,CUAN,TCPI,INKP,UNTR,ICBP'

In [ ]:
def transform_json_to_df(api_url, symbol, api_key):
  API_URL = api_url
  SYMBOL = symbol
  API_KEY = api_key

  query_params = urllib.parse.urlencode({'symbols': SYMBOL})
  new_url = API_URL + '?' + query_params

  url = new_url
  headers = {"X-API-KEY": API_KEY}

  response = requests.get(url, headers=headers).json()
  response_str = json.dumps(response)
  df = pd.json_normalize(json.loads(response_str), record_path=['data', 'results'])
  df = df[['company.name', 'company.logo', 'symbol', 'date', 'open', 'high', 'low', 'close', 'volume']]
  df["hasil_mean"] = df.apply(lambda x: (x["open"] + x["high"] + x["low"] + x["close"]) / 4, axis=1)
  return df

In [ ]:
df_json = transform_json_to_df(API_URL, SYMBOL_string, API_KEY)

In [ ]:
df_json

,company.name,company.logo,symbol,date,open,high,low,close,volume,hasil_mean
0,Adaro Minerals Indonesia Tbk.,https://s3.goapi.id/logo/ADMR.jpg,ADMR,2023-12-08,1305,1400,1305,1370,50683400,1345.00
1,Adaro Energy Indonesia Tbk.,https://s3.goapi.id/logo/ADRO.jpg,ADRO,2023-12-08,2570,2600,2550,2560,41613300,2570.00
2,AKR Corporindo Tbk.,https://s3.goapi.id/logo/AKRA.jpg,AKRA,2023-12-08,1340,1405,1340,1395,46031900,1370.00
3,Amman Mineral Internasional Tbk.,https://s3.goapi.id/logo/AMMN.jpg,AMMN,2023-12-08,6475,6500,6300,6450,61148900,6431.25
4,Sumber Alfaria Trijaya Tbk.,https://s3.goapi.id/logo/AMRT.jpg,AMRT,2023-12-08,2890,2890,2770,2850,30778700,2850.00
5,Aneka Tambang Tbk.,https://s3.goapi.id/logo/ANTM.jpg,ANTM,2023-12-08,1650,1690,1645,1685,41337300,1667.50
6,Pacific Strategic Financial Tbk.,https://s3.goapi.id/logo/APIC.jpg,APIC,2023-12-08,1175,1180,1150,1160,280643000,1166.25
7,Archi Indonesia Tbk.,https://s3.goapi.id/logo/ARCI.jpg,ARCI,2023-12-08,400,484,394,452,334145700,432.50
8,Bank Jago Tbk.,https://s3.goapi.id/logo/ARTO.jpg,ARTO,2023-12-08,3040,3220,2960,3180,40221100,3100.00
9,Astra International Tbk.,https://s3.goapi.id/logo/ASII.jpg,ASII,2023-12-08,5675,5700,5625,5675,26972000,5668.75


In [ ]:
df_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   company.name  35 non-null     object 
 1   company.logo  35 non-null     object 
 2   symbol        35 non-null     object 
 3   date          35 non-null     object 
 4   open          35 non-null     int64  
 5   high          35 non-null     int64  
 6   low           35 non-null     int64  
 7   close         35 non-null     int64  
 8   volume        35 non-null     int64  
 9   hasil_mean    35 non-null     float64
dtypes: float64(1), int64(5), object(4)
memory usage: 2.9+ KB


In [ ]:
df_json.describe()

,open,high,low,close,volume,hasil_mean
count,35.000000,35.000000,35.000000,35.000000,3.500000e+01,35.000000
mean,4452.714286,4610.714286,4383.142857,4521.342857,5.477839e+08,4491.978571
std,4271.012613,4391.532250,4242.701700,4366.605142,2.196451e+09,4316.725864
min,97.000000,106.000000,94.000000,104.000000,5.209000e+06,102.250000
25%,1322.500000,1402.500000,1322.500000,1382.500000,2.716685e+07,1357.500000
50%,3600.000000,3740.000000,3470.000000,3680.000000,4.603190e+07,3622.500000
75%,6112.500000,6137.500000,5975.000000,6075.000000,1.898722e+08,6075.000000
max,21800.000000,22350.000000,21725.000000,22300.000000,1.302449e+10,22043.750000


In [ ]:
df_json.isnull().sum()

company.name    0
company.logo    0
symbol          0
date            0
open            0
high            0
low             0
close           0
volume          0
hasil_mean      0
dtype: int64

In [ ]:
df_json.duplicated().sum()

0

### Model Content-based Recommendation
Merekomendasikan item yang mirip dengan item similarity degree. Content-based filtering akan mempelajari data saham pada mean berdasarkan objek data similarity degree.

In [ ]:
# Inisialisasi TfidfVectorizer
tfv = TfidfVectorizer()
# Melakukan perhitungan idf pada data cuisine
tfv.fit(df_json['hasil_mean'].astype(str))

TfidfVectorizer()

Selanjutnya, melakukan fit dan transformasi atribut data saham yaitu hasil_mean ke dalam bentuk matriks.

In [ ]:
# Melakukan fit lalu ditransformasikan ke bentuk matrix
tfidf_matrix = tfv.fit_transform(df_json['hasil_mean'].astype(str))

# Melihat ukuran matrix tfidf
tfidf_matrix.shape

(35, 37)

Untuk menghasilkan vektor tf-idf dalam bentuk matriks, menggunakan fungsi [todense()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.todense.html)

In [ ]:
tfidf_matrix.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:

# Membuat dataframe untuk melihat tf-idf matrix
# Kolom diisi dengan hasil mean
# Baris diisi dengan kode saham

pd.DataFrame(
    tfidf_matrix.todense(),
    columns = tfv.get_feature_names_out(),
    index   = df_json.symbol
).sample(20, axis=1).sample(10, axis=0)

,1166,5668,2717,2570,1147,1345,5143,1713,4650,3930,5718,138,1667,102,8262,8787,5406,22043,4352,103
symbol,,,,,,,,,,,,,,,,,,,,
BBRI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.812455,0.0,0.0,0.0
GOTO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.812455,0.0,0.0,0.000000,0.0,0.0,0.0
PGEO,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
BREN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
NATO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
INKP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.000000,0.0,0.0,0.0
MDKA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
AKRA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
ADRO,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0


### Cosine Similiarity
Selanjutnya, menghitung derajat kesamaan (similarity degree) antar hasil mean dengan teknik cosine similarity menggunakan fungsi [cosine_similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) dari library sklearn.

In [ ]:
# Menghitung cosine similarity pada matrix tf-idf
cosine_sim = cosine_similarity(tfidf_matrix)
cosine_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
# Membuat dataframe dari variabel cosine_sim dengan baris dan kolom berupa kode saham
cosine_sim_df = pd.DataFrame(
    cosine_sim,
    columns=df_json['symbol'],
    index=df_json['symbol']
)

print('Shape:', cosine_sim_df.shape)

# Melihat similarity matrix pada setiap hasil mean
cosine_sim_df.sample(5, axis=1).sample(10, axis=0)

Shape: (35, 35)


symbol,FILM,BBNI,ICBP,TCPI,ASII
symbol,,,,,
GZCO,0.0,0.0,0.0,0.0,0.0
APIC,0.0,0.0,0.0,0.0,0.0
ICBP,0.0,0.0,1.0,0.0,0.0
BREN,0.0,0.0,0.0,0.0,0.0
TLKM,0.0,0.0,0.0,0.0,0.0
ADRO,0.0,0.0,0.0,0.0,0.0
BBRI,0.0,0.0,0.0,0.0,0.0
TPIA,0.0,0.0,0.0,0.0,0.0
FILM,1.0,0.0,0.0,0.0,0.0


In [ ]:
def model_function(df_json):
  # Inisialisasi TfidfVectorizer
  tfv = TfidfVectorizer()
  # Melakukan perhitungan idf pada data cuisine
  tfv.fit(df_json['hasil_mean'].astype(str))
  tfidf_matrix = tfv.fit_transform(df_json['hasil_mean'].astype(str))
  tfidf_matrix.todense()
  cosine_sim = cosine_similarity(tfidf_matrix)
  cosine_sim_df = pd.DataFrame(
    cosine_sim,
    columns=df_json['symbol'],
    index=df_json['symbol']
  )
  return cosine_sim_df

### Recommendation Testing

In [ ]:
cosine_sim_df

symbol,ADMR,ADRO,AKRA,AMMN,AMRT,ANTM,APIC,ARCI,ARTO,ASII,...,MEDC,MIKA,NATO,PANI,PGEO,PTRO,TCPI,TLKM,TPIA,UNTR
symbol,,,,,,,,,,,,,,,,,,,,,
ADMR,1.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
ADRO,0.0,1.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
AKRA,0.0,0.0,1.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
AMMN,0.0,0.0,0.0,1.000000,0.0,0.0,0.339917,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
AMRT,0.0,0.0,0.0,0.000000,1.0,0.0,0.000000,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
ANTM,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
APIC,0.0,0.0,0.0,0.339917,0.0,0.0,1.000000,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
ARCI,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,1.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
ARTO,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,1.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000


Membuat fungsi saham_recommendations untuk menampilkan data kode saham yang direkomendasikan oleh algoritma sistem yang telah dibuat, dengan parameter masukan berupa kode saham yang akan di dinisiasi pada similarity degree.

In [ ]:

def saham_recommendations(hasil_mean, similarity_data=cosine_sim_df, items=df_json, k=10):
    # Mengambil data dengan menggunakan argpartition untuk melakukan partisi secara tidak langsung sepanjang sumbu yang diberikan
    # Dataframe diubah menjadi numpy
    # Range(start, stop, step)
    if hasil_mean == "Tidak ada rekomendasi Saham, Pengeluaran anda terlalu banyak":
      return "Tidak ada rekomendasi Saham, Pengeluaran anda terlalu banyak"
    else:
      index = similarity_data.loc[:,hasil_mean].to_numpy().argpartition(range(-1, -k, -1))
      # Mengambil data dengan similarity terbesar dari index yang ada
      closest = similarity_data.columns[index[-1:-(k+2):-1]]

      return pd.DataFrame(closest).merge(items).head(k).drop_duplicates()

In [ ]:
df_json['symbol']

0     ADMR
1     ADRO
2     AKRA
3     AMMN
4     AMRT
5     ANTM
6     APIC
7     ARCI
8     ARTO
9     ASII
10    BBCA
11    BBNI
12    BBRI
13    BIPI
14    BMRI
15    BREN
16    BRPT
17    CUAN
18    FILM
19    GOTO
20    GZCO
21    ICBP
22    INCO
23    INKP
24    MDKA
25    MEDC
26    MIKA
27    NATO
28    PANI
29    PGEO
30    PTRO
31    TCPI
32    TLKM
33    TPIA
34    UNTR
Name: symbol, dtype: object

In [ ]:
df_json['hasil_mean']

0      1345.00
1      2570.00
2      1370.00
3      6431.25
4      2850.00
5      1667.50
6      1166.25
7       432.50
8      3100.00
9      5668.75
10     8787.50
11     5143.75
12     5406.25
13      103.75
14     5718.75
15     7931.25
16     1713.75
17    10300.00
18     3622.50
19      102.25
20      138.50
21    10825.00
22     4352.50
23     8262.50
24     2510.00
25     1030.00
26     2717.50
27      183.50
28     4685.00
29     1147.50
30     7125.00
31     8187.50
32     3930.00
33     4650.00
34    22043.75
Name: hasil_mean, dtype: float64

In [ ]:
def transform_data(data_pemasukan, data_pengeluaran, data_saham):

  # Inisialisasi variabel testing
  # data pemasukan dan pengeluaran selama seminggu
  pemasukan_mean = np.sum(data_pemasukan) / len(data_pemasukan)
  pengeluaran_mean = np.sum(data_pengeluaran).mean() / len(data_pengeluaran)
  perbandingan = pemasukan_mean - pengeluaran_mean

  if (perbandingan < 0):
    return "Tidak ada rekomendasi Saham, Pengeluaran anda terlalu banyak"
  else:
    # Cari baris yang sesuai
    diff = perbandingan - data_saham["hasil_mean"]

    idx = diff <= 10

    actual_df = data_saham[idx]
    data_selected = actual_df['hasil_mean'].min()

    saham_selected = data_saham[data_saham.hasil_mean.eq(data_selected)]

    data_selected_saham = saham_selected.loc[:, 'symbol'].to_string(index=False)
    return data_selected_saham

In [ ]:
data_pemasukan = [70000, 0, 0, 0, 0, 0, 0]
data_pengeluaran = [50000, 0, 0, 0, 0, 0, 0]

data_selected_saham = transform_data(data_pemasukan, data_pengeluaran, df_json)

In [ ]:
data_selected_saham

'AMRT'

### Hasil rekomendasi

In [ ]:
saham_recommendations(data_selected_saham)

,symbol,company.name,company.logo,date,open,high,low,close,volume,hasil_mean
0,AMRT,Sumber Alfaria Trijaya Tbk.,https://s3.goapi.id/logo/AMRT.jpg,2023-12-08,2890,2890,2770,2850,30778700,2850.00
1,UNTR,United Tractors Tbk.,https://s3.goapi.id/logo/UNTR.jpg,2023-12-08,21800,22350,21725,22300,6433900,22043.75
2,BBRI,Bank Rakyat Indonesia (Persero) Tbk.,https://s3.goapi.id/logo/BBRI.jpg,2023-12-08,5425,5450,5375,5375,130542600,5406.25
3,BBNI,Bank Negara Indonesia (Persero) Tbk.,https://s3.goapi.id/logo/BBNI.jpg,2023-12-08,5200,5225,5075,5075,47842200,5143.75
4,BBCA,Bank Central Asia Tbk.,https://s3.goapi.id/logo/BBCA.jpg,2023-12-08,8800,8850,8750,8750,43891600,8787.50
5,BREN,Barito Renewables Energy Tbk.,https://s3.goapi.id/logo/BREN.jpg,2023-12-08,7750,8175,7750,8050,50224300,7931.25
6,BMRI,Bank Mandiri (Persero) Tbk.,https://s3.goapi.id/logo/BMRI.jpg,2023-12-08,5750,5775,5650,5700,98036690,5718.75
7,BIPI,Astrindo Nusantara Infrastruktur Tbk.,https://s3.goapi.id/logo/BIPI.jpg,2023-12-08,104,106,101,104,582602200,103.75
8,BRPT,Barito Pacific Tbk.,https://s3.goapi.id/logo/BRPT.jpg,2023-12-08,1625,1950,1530,1750,1376205000,1713.75
9,ADRO,Adaro Energy Indonesia Tbk.,https://s3.goapi.id/logo/ADRO.jpg,2023-12-08,2570,2600,2550,2560,41613300,2570.00


10 Saham (sesuai banyak k) yang direkomendasikan bersesuaian dengan kode saham dari Cosine Similiarity

akurasi nya 10/10 * 100 = 100%

### Test deploy

In [ ]:
!pip install dill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.7 MB/s eta 0:00:00


In [ ]:
import dill

In [ ]:
C = 1
output_file = f'model_recomendation={C}.bin'

In [ ]:
f_out = open(output_file, 'wb')
dill.dump((generate_saham_tren, transform_json_to_df, transform_data, model_function), f_out)
f_out.close()

In [ ]:
!ls -lh *.bin

-rw-r--r-- 1 root root 2.4K Dec 10 11:24 'model_recomendation=1.bin'


Save the modeland test with new session

In [ ]:
!pip install dill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 1.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import json
import requests
import urllib.parse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random
import dill

In [ ]:
API_URL = "https://api.goapi.io/stock/idx/prices"
API_KEY = '833b2587-34b1-5172-801a-6be0cf21'
SYMBOL_URL = "https://api.goapi.io/stock/idx/trending"

In [ ]:
input_file = 'model_recomendation=1 (2).bin'

In [ ]:
with open(input_file, 'rb') as f_in:
    generate_saham_tren, transform_json_to_df, transform_data, model = dill.load(f_in)

In [ ]:
SYMBOL_string = generate_saham_tren(SYMBOL_URL, API_KEY)

In [ ]:
SYMBOL_string

'GOTO,GZCO,BRPT,BIPI,PGEO,NATO,ARCI,APIC,MEDC,TPIA,BBRI,BMRI,TLKM,AMMN,ADMR,BREN,BBNI,AKRA,BBCA,PANI,ADRO,ANTM,ARTO,MDKA,AMRT,PTRO,ASII,MIKA,INCO,FILM,CUAN,TCPI,INKP,UNTR,ICBP'

In [ ]:
data_saham = transform_json_to_df(API_URL, SYMBOL_string, API_KEY)

In [ ]:
data_saham

,company.name,company.logo,symbol,date,open,high,low,close,volume,hasil_mean
0,Adaro Minerals Indonesia Tbk.,https://s3.goapi.id/logo/ADMR.jpg,ADMR,2023-12-08,1305,1400,1305,1370,50683400,1345.00
1,Adaro Energy Indonesia Tbk.,https://s3.goapi.id/logo/ADRO.jpg,ADRO,2023-12-08,2570,2600,2550,2560,41613300,2570.00
2,AKR Corporindo Tbk.,https://s3.goapi.id/logo/AKRA.jpg,AKRA,2023-12-08,1340,1405,1340,1395,46031900,1370.00
3,Amman Mineral Internasional Tbk.,https://s3.goapi.id/logo/AMMN.jpg,AMMN,2023-12-08,6475,6500,6300,6450,61148900,6431.25
4,Sumber Alfaria Trijaya Tbk.,https://s3.goapi.id/logo/AMRT.jpg,AMRT,2023-12-08,2890,2890,2770,2850,30778700,2850.00
5,Aneka Tambang Tbk.,https://s3.goapi.id/logo/ANTM.jpg,ANTM,2023-12-08,1650,1690,1645,1685,41337300,1667.50
6,Pacific Strategic Financial Tbk.,https://s3.goapi.id/logo/APIC.jpg,APIC,2023-12-08,1175,1180,1150,1160,280643000,1166.25
7,Archi Indonesia Tbk.,https://s3.goapi.id/logo/ARCI.jpg,ARCI,2023-12-08,400,484,394,452,334145700,432.50
8,Bank Jago Tbk.,https://s3.goapi.id/logo/ARTO.jpg,ARTO,2023-12-08,3040,3220,2960,3180,40221100,3100.00
9,Astra International Tbk.,https://s3.goapi.id/logo/ASII.jpg,ASII,2023-12-08,5675,5700,5625,5675,26972000,5668.75


In [ ]:
model = model(data_saham)

In [ ]:
model

symbol,ADMR,ADRO,AKRA,AMMN,AMRT,ANTM,APIC,ARCI,ARTO,ASII,...,MEDC,MIKA,NATO,PANI,PGEO,PTRO,TCPI,TLKM,TPIA,UNTR
symbol,,,,,,,,,,,,,,,,,,,,,
ADMR,1.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
ADRO,0.0,1.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
AKRA,0.0,0.0,1.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
AMMN,0.0,0.0,0.0,1.000000,0.0,0.0,0.339917,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
AMRT,0.0,0.0,0.0,0.000000,1.0,0.0,0.000000,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
ANTM,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
APIC,0.0,0.0,0.0,0.339917,0.0,0.0,1.000000,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
ARCI,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,1.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
ARTO,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,1.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000


In [ ]:
data_pemasukan = [70000, 0, 0, 0, 0, 0, 0]
data_pengeluaran = [50000, 0, 0, 0, 0, 0, 0]

test = transform_data(data_pemasukan, data_pengeluaran, data_saham)

In [ ]:
test

'AMRT'

In [ ]:

def saham_recommendations(hasil_mean, similarity_data=model, items=data_saham, k=10):
    # Mengambil data dengan menggunakan argpartition untuk melakukan partisi secara tidak langsung sepanjang sumbu yang diberikan
    # Dataframe diubah menjadi numpy
    # Range(start, stop, step)
    if hasil_mean == "Tidak ada rekomendasi Saham, Pengeluaran anda terlalu banyak":
      return "Tidak ada rekomendasi Saham, Pengeluaran anda terlalu banyak"
    else:
      index = similarity_data.loc[:,hasil_mean].to_numpy().argpartition(range(-1, -k, -1))
      # Mengambil data dengan similarity terbesar dari index yang ada
      closest = similarity_data.columns[index[-1:-(k+2):-1]]

      return pd.DataFrame(closest).merge(items).head(k).drop_duplicates()

In [ ]:
saham_recommendations(test)

,symbol,company.name,company.logo,date,open,high,low,close,volume,hasil_mean
0,AMRT,Sumber Alfaria Trijaya Tbk.,https://s3.goapi.id/logo/AMRT.jpg,2023-12-08,2890,2890,2770,2850,30778700,2850.00
1,UNTR,United Tractors Tbk.,https://s3.goapi.id/logo/UNTR.jpg,2023-12-08,21800,22350,21725,22300,6433900,22043.75
2,BBRI,Bank Rakyat Indonesia (Persero) Tbk.,https://s3.goapi.id/logo/BBRI.jpg,2023-12-08,5425,5450,5375,5375,130542600,5406.25
3,BBNI,Bank Negara Indonesia (Persero) Tbk.,https://s3.goapi.id/logo/BBNI.jpg,2023-12-08,5200,5225,5075,5075,47842200,5143.75
4,BBCA,Bank Central Asia Tbk.,https://s3.goapi.id/logo/BBCA.jpg,2023-12-08,8800,8850,8750,8750,43891600,8787.50
5,BREN,Barito Renewables Energy Tbk.,https://s3.goapi.id/logo/BREN.jpg,2023-12-08,7750,8175,7750,8050,50224300,7931.25
6,BMRI,Bank Mandiri (Persero) Tbk.,https://s3.goapi.id/logo/BMRI.jpg,2023-12-08,5750,5775,5650,5700,98036690,5718.75
7,BIPI,Astrindo Nusantara Infrastruktur Tbk.,https://s3.goapi.id/logo/BIPI.jpg,2023-12-08,104,106,101,104,582602200,103.75
8,BRPT,Barito Pacific Tbk.,https://s3.goapi.id/logo/BRPT.jpg,2023-12-08,1625,1950,1530,1750,1376205000,1713.75
9,ADRO,Adaro Energy Indonesia Tbk.,https://s3.goapi.id/logo/ADRO.jpg,2023-12-08,2570,2600,2550,2560,41613300,2570.00


### Test Flask

In [ ]:
# !pip install flask

In [ ]:
# !pip install pyngrok

In [ ]:
# !ngrok authtoken 2Z5qKCbKRO5vainLb7Rwi2vWF9s_2sPbwVc5KEaNQt4yZwoFz

In [ ]:
# global data_selected_saham
# # Import library
# from flask import Flask
# from pyngrok import ngrok
# import json

# import os
# import threading

# # Inisialisasi Flask
# app = Flask(__name__)
# port = "5000"

# public_url = ngrok.connect(port).public_url
# print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# @app.route('/')
# def index():
#   return 'hello world'


# @app.route('/saham')
# def recommendations():
#     # Muat model
#     with open('model.pkl', 'rb') as f:
#         cosine_sim_df = pickle.load(f)

#     # Lakukan prediksi
#     recommendations = saham_recommendations(data_selected_saham, similarity_data=cosine_sim_df)

#     # Kembalikan hasil prediksi
#     return json.dumps(recommendations.to_dict(orient='records'))

# # Jalankan Flask
# if __name__ == '__main__':
#     threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()